## Allocating Elastic IP address

In [1]:
import boto3

AWS_REGION = "us-east-1"
VPC_NAME = 'Lab-1'
EC2_CLIENT = boto3.client('ec2', region_name=AWS_REGION)
EC2_RESOURSE = boto3.resource("ec2", region_name=AWS_REGION)

ElasticIP = EC2_CLIENT.allocate_address( Domain='vpc',
                                    NetworkBorderGroup=AWS_REGION,
                                    TagSpecifications=[{
                                            'ResourceType': 'elastic-ip',
                                            'Tags': [{
                                                        'Key': 'Name',
                                                        'Value': VPC_NAME
                                                    }]
                                        },
                                    ])

ElasticIP

dict_keys(['PublicIp', 'AllocationId', 'PublicIpv4Pool', 'NetworkBorderGroup', 'Domain', 'ResponseMetadata'])

## Create Internet Gateway

In [2]:
internet_gateway = EC2_CLIENT.create_internet_gateway(TagSpecifications=[
                    {
                        'ResourceType': 'internet-gateway',
                        'Tags': [
                            {
                                'Key': 'Name',
                                'Value': VPC_NAME
                            },
                        ]
                    },
                ])

In [3]:
internet_gateway

{'InternetGateway': {'Attachments': [],
  'InternetGatewayId': 'igw-02cf9dcb17363df0f',
  'OwnerId': '593222115585',
  'Tags': [{'Key': 'Name', 'Value': 'Lab-1'}]},
 'ResponseMetadata': {'RequestId': 'b011113a-bc12-46c6-9d5a-b6c3a7926639',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b011113a-bc12-46c6-9d5a-b6c3a7926639',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '562',
   'date': 'Fri, 04 Feb 2022 17:24:48 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Creating a Custom VPC

In [4]:
vpc = EC2_CLIENT.create_vpc(CidrBlock='10.0.0.0/16',
                            AmazonProvidedIpv6CidrBlock=False,
                                    InstanceTenancy='default',
                                    TagSpecifications=[{
                                        'ResourceType':'vpc',
                                        'Tags': [{
                                                'Key':'Name',
                                                'Value':VPC_NAME
                                                }]
                                    }])

EC2_CLIENT.get_waiter('vpc_available').wait(VpcIds=[vpc['Vpc']['VpcId']])

In [24]:
EC2_CLIENT.modify_vpc_attribute(VpcId=vpc['Vpc']['VpcId'], EnableDnsHostnames={'Value': True})
EC2_CLIENT.modify_vpc_attribute(VpcId=vpc['Vpc']['VpcId'], EnableDnsSupport={'Value': True})

{'ResponseMetadata': {'RequestId': 'c1134a68-651a-47bf-befb-413d45aa8e88',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c1134a68-651a-47bf-befb-413d45aa8e88',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Fri, 04 Feb 2022 18:23:19 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [6]:
vpc

{'Vpc': {'CidrBlock': '10.0.0.0/16',
  'DhcpOptionsId': 'dopt-7aebe400',
  'State': 'pending',
  'VpcId': 'vpc-064a203c4c15fe796',
  'OwnerId': '593222115585',
  'InstanceTenancy': 'default',
  'Ipv6CidrBlockAssociationSet': [],
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-0ca51046681ac79d6',
    'CidrBlock': '10.0.0.0/16',
    'CidrBlockState': {'State': 'associated'}}],
  'IsDefault': False,
  'Tags': [{'Key': 'Name', 'Value': 'Lab-1'}]},
 'ResponseMetadata': {'RequestId': '73088001-0cbc-481d-b2df-4854ef235e69',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '73088001-0cbc-481d-b2df-4854ef235e69',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1048',
   'date': 'Fri, 04 Feb 2022 17:24:49 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Attach Internet Gateway to a VPC

In [16]:
internet_gateway = EC2_CLIENT.attach_internet_gateway(
                                                InternetGatewayId=internet_gateway['InternetGateway']['InternetGatewayId'], 
                                                VpcId=vpc['Vpc']['VpcId']
                                                )          

## Creating a Custom Subnet

In [17]:
subnets = [{'Name':'Public Subnet 1',
        'AvailabilityZone': AWS_REGION +'a',
        'CidrBlock':'10.0.0.0/24'},
        {'Name':'Private Subnet 1',
        'AvailabilityZone': AWS_REGION +'a',
        'CidrBlock':'10.0.1.0/24'},
        {'Name':'Public Subnet 2',
        'AvailabilityZone':AWS_REGION +'b',
        'CidrBlock':'10.0.2.0/24'},
        {'Name':'Private Subnet 2',
        'AvailabilityZone':AWS_REGION +'b',
        'CidrBlock':'10.0.3.0/24'}]

In [18]:
subnet_ids = []
for subnet in subnets:
    res = EC2_CLIENT.create_subnet(TagSpecifications=[{
                                                'ResourceType': 'subnet',
                                                'Tags': [{
                                                    'Key': 'Name',
                                                    'Value': subnet['Name']
                                                }]
                                                }],
                                                AvailabilityZone=subnet['AvailabilityZone'],
                                                VpcId=vpc['Vpc']['VpcId'],
                                                CidrBlock=subnet['CidrBlock'])
    subnet_ids.append(res['Subnet']['SubnetId'])

## Create NAT Gateway

In [19]:
nat_gateway = EC2_CLIENT.create_nat_gateway(
                                        ConnectivityType='public',
                                        AllocationId=ElasticIP['AllocationId'],
                                        SubnetId=subnet_ids[0],
                                        TagSpecifications=[{
                                            'ResourceType':'natgateway',
                                            'Tags': [{
                                                'Key': 'Name',
                                                'Value': VPC_NAME
                                            }]
                                        }])

EC2_CLIENT.get_waiter('nat_gateway_available').wait(NatGatewayIds=[nat_gateway['NatGateway']['NatGatewayId']])

In [20]:
Private_Route_Table = EC2_CLIENT.describe_route_tables(
                                        Filters=[{
                                                'Name': 'vpc-id',
                                                'Values': [vpc['Vpc']['VpcId']]
                                            },
                                            {
                                                'Name': 'association.main',
                                                'Values': ['true']
                                            }
                                            ]
                                        )
Private_Route_Table['RouteTables']

[{'Associations': [{'Main': True,
    'RouteTableAssociationId': 'rtbassoc-0c42f652e914b0e99',
    'RouteTableId': 'rtb-07e816fbcbecefeaf',
    'AssociationState': {'State': 'associated'}}],
  'PropagatingVgws': [],
  'RouteTableId': 'rtb-07e816fbcbecefeaf',
  'Routes': [{'DestinationCidrBlock': '10.0.0.0/16',
    'GatewayId': 'local',
    'Origin': 'CreateRouteTable',
    'State': 'active'}],
  'Tags': [],
  'VpcId': 'vpc-064a203c4c15fe796',
  'OwnerId': '593222115585'}]

In [21]:
EC2_CLIENT.associate_route_table(
                                RouteTableId=Private_Route_Table['RouteTables'][0]['Associations'][0]['RouteTableId'],
                                SubnetId=subnet_ids[1]
                            )
EC2_CLIENT.associate_route_table(
                                RouteTableId=Private_Route_Table['RouteTables'][0]['Associations'][0]['RouteTableId'],
                                SubnetId=subnet_ids[3]
                            )

{'AssociationId': 'rtbassoc-0963d811313d81a10',
 'AssociationState': {'State': 'associated'},
 'ResponseMetadata': {'RequestId': '95a79a55-d67b-4085-ac54-f3f636c14efd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '95a79a55-d67b-4085-ac54-f3f636c14efd',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '356',
   'date': 'Fri, 04 Feb 2022 17:46:58 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [22]:
Public_Route_Table = EC2_CLIENT.create_route_table(
                                        VpcId=vpc['Vpc']['VpcId'],
                                        TagSpecifications=[
                                            {
                                                'ResourceType': 'route-table',
                                                'Tags': [
                                                    {
                                                        'Key': 'Name',
                                                        'Value': 'Private Route Table'
                                                    }
                                                ]
                                            },
                                        ])
Public_Route_Table

{'RouteTable': {'Associations': [],
  'PropagatingVgws': [],
  'RouteTableId': 'rtb-055228ce9e159f1f7',
  'Routes': [{'DestinationCidrBlock': '10.0.0.0/16',
    'GatewayId': 'local',
    'Origin': 'CreateRouteTable',
    'State': 'active'}],
  'Tags': [{'Key': 'Name', 'Value': 'Private Route Table'},
   {'Key': 'Main', 'Value': 'No'}],
  'VpcId': 'vpc-064a203c4c15fe796',
  'OwnerId': '593222115585'},
 'ResponseMetadata': {'RequestId': '83b00bbb-4779-4913-881a-0e5af2137e9d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '83b00bbb-4779-4913-881a-0e5af2137e9d',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '987',
   'date': 'Fri, 04 Feb 2022 17:55:19 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [23]:
EC2_CLIENT.associate_route_table(
                                RouteTableId=Public_Route_Table['RouteTable']['RouteTableId'],
                                SubnetId=subnet_ids[0]
                            )
EC2_CLIENT.associate_route_table(
                                RouteTableId=Public_Route_Table['RouteTable']['RouteTableId'],
                                SubnetId=subnet_ids[2]
                            )

{'AssociationId': 'rtbassoc-0dcccf122dfe22988',
 'AssociationState': {'State': 'associated'},
 'ResponseMetadata': {'RequestId': '9680b573-8b8a-4d9a-a7e3-45cb9a9608f2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9680b573-8b8a-4d9a-a7e3-45cb9a9608f2',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '356',
   'date': 'Fri, 04 Feb 2022 17:59:09 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Create Security Group

In [26]:
GROUP_NAME = 'Web Security Group'
security_group = EC2_CLIENT.create_security_group(Description='Enable HTTP access',
                                                      GroupName=GROUP_NAME,
                                                      VpcId=vpc['Vpc']['VpcId'],
                                                      TagSpecifications=[{
                                                                        'ResourceType':'security-group',
                                                                        'Tags': [{
                                                                                    'Key':'Name',
                                                                                    'Value':GROUP_NAME
                                                                                    }]
                                                                         }]
                                                    )
security_group

ClientError: An error occurred (UnauthorizedOperation) when calling the CreateSecurityGroup operation: You are not authorized to perform this operation. Encoded authorization failure message: zXW69URMtMe-82Kh2YVwxH7t1cgmncmAGVXbfT5UysOiW9nPyLRYv6v9KQL1bLpP5d39pb6A0k1LDB1VgO_bmP70S36hbDTnbzjnnosswg5FKEe3Dhgp06UuGmVkDk-nuAEh9b1-kQyyQdr8xkDYTNVBAlC91h1MZttsZM16-l9BQ3DzeoUJI0MVUc0qlYAjJ_zAPDfWtuGx4w8TVr5O45-tDE6rqhVKoJojzyMgFzVUNY_G598MuIoLGuhoG1Qhni82-EREnhgylr_XiHvr1Sm108uU4ICWh2QPmsu8ielcndoDO0lZJo2Trj9WRC2dtk_yER_BhYPLe5fNvwcRCEn0Kk-jM-_0UV0M4V-IPmKf_N8QaeqIxbZJ0Y1TkuAygrMZH6DaFSuYoWxQPI4N7Npmdj7gplVkBeWyGBLCV0wOhimBIyQrIdT-LHMcXEKBxew__bYO0nAH0YcyzWafkt25r4kgg6xoS-UdlDacc1kxZdHuxXOeLQCPrA09aaiLx1Zfld269YYMpMtC385VQarWzLVoBzPDzLGVR0mr7GsDFLwhEkMl0PEjG-cZg30sQzauPR3qjO4zcJbS8VI-BkepnQ6aNkqdoxv3oQzihclJ-G2cbi5TA3uMq_PANVmrq1wNGRGO5eX0TTMX4tSNwlj1-2sdDTefv8-ZhtN1XYbeFghTHTQsWWtKyjKxwG391tobFj49f_SyTVHi-d9R-lDdbj51jBW66EZxVIy49SRh_UYFDU_r4Q9IwBsVmA

In [ ]:
response = EC2_CLIENT.authorize_security_group_ingress(
                                                    GroupId=security_group_id,
                                                    IpPermissions=[{
                                                                    'IpProtocol': 'tcp',
                                                                    'FromPort': 80,
                                                                    'ToPort': 80,
                                                                    'IpRanges': [{
                                                                                    'CidrIp': '0.0.0.0/0'
                                                                                }]
                                                                    }]
                                                    )